In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import math

from sklearn.datasets import fetch_20newsgroups
import numpy as np

import argparse
from types import SimpleNamespace

import random
random.seed(1234)

import pandas as pd
import matplotlib.pyplot as plt

from nltk.tokenize import RegexpTokenizer

import bsg, tools

In [6]:
# Get corpus: 20 news groups

newsgroups_train = fetch_20newsgroups(subset='train')

window = 2

vocabulary, tokenized_corpus, word2idx, idx2word = tools.tokenize_corpus(newsgroups_train.data)

indexed_corpus = tools.index_tokenized_corpus(tokenized_corpus, word2idx)

corpus_center_words, corpus_context_words = tools.get_corpus_centers_contexts(indexed_corpus, window)

unigram = tools.make_unigram_dict(corpus_center_words)

In [15]:
args_dict = {"vocab_size" : len(vocabulary), "window" : window, 
             "batch_size" : 200, "optimizer" : 80, "learning_rate" : 0.0005, 
             "momentum" : 0.99, "num_epoch" : 50, "init_mult" : 1, 
             "variance" : 0.995, "start" : True, "nogpu" : True, 
             "embedding_dim" : 300, "freeze" : False}

args = SimpleNamespace(**args_dict)

In [ ]:
model = bsg.BSG(args.window, unigram, args.vocab_size)
optimizer = torch.optim.Adam(model.parameters(), args.learning_rate, betas=(args.momentum, 0.999))
model = bsg.train(model, args, optimizer, corpus_center_words[0:1000], corpus_context_words[0:1000])



Epoch 0, loss=233587.828125
Epoch 5, loss=168299.25


In [14]:
print(len(corpus_center_words))
print(len(corpus_center_words[0:1000].unique()))

2539447
526


In [ ]:
idx2word(corpus_center_words[0])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim_matrix = cosine_similarity(model_GSMLDA2.word_embedding.weight.detach().numpy(), 
                                   model_GSMLDA2.word_embedding.weight.detach().numpy())
def n_closest_words(word, cos_sim_matrix, n):
    word_index = vectorizer.get_feature_names().index(word)
    close_words_indices = np.argsort(cos_sim_matrix[word_index])[-n:]
    print(close_words_indices)
    return [vectorizer.get_feature_names()[j] for j in close_words_indices]

n_closest_words("nasa", cos_sim_matrix, 20)